#  MLP Classifier for  SER (Speech Emotion Recognition)

Multilayer perceptrons (MLP) are often applied to supervised learning problems. They train on a set of input-output pairs and learn to model the correlation (or dependencies) between those inputs and outputs. The networkthus has a simple interpretation as a form of input-output model, with the weights and thresholds (biases) the free parameters of the model.

## Dataset

The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS),and it is free to download. This dataset has 7356 files rated by 247 individuals 10 times on emotional validity, intensity, and genuineness. here the Speech files of all actors (01-24) will be used and the files are available under path ../datasets/RAVDESS. It contains 1440 files: 60 trials per actor x 24 actors = 1440. 
Filename identifiers:
<ol>
<li>Modality (01 = full-AV, 02 = video-only, 03 = audio-only).</li>
<li>Vocal channel (01 = speech, 02 = song).</li>
<li>Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).</li>
<li>Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.</li>
<li>Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").</li>
<li>Repetition (01 = 1st repetition, 02 = 2nd repetition)..</li>
<li>Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).</li>
</ol>

## Downloading Libraries

In [1]:
#!pip install librosa soundfile numpy sklearn pyaudio

In [2]:
#!pip install soundfile

## Libraries Import

In [1]:
# Import libraries
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import os, glob, pickle
import IPython.display as ipd 
from tqdm import tqdm

import soundfile
import seaborn as sns
import pandas as pd

# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint


# sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

## Feature Extraction

Defining a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file name and three Boolean parameters for the three features:
file.
<ol>
<li>file: for the given file name with the path</li>
<li>mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound</li>
<li>chroma: Pertains to the 12 different pitch classes</li>
<li>mel: Mel Spectrogram Frequency</li>
</ol>




In [2]:
#Extract features (mfcc, chroma, mel) from a given sound file(with path)
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#Emotions in the RAVDESS dataset, it will not take the neutral into the modeling process
emotions={
  '01':'neutral',
  '02':'neutral',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprise'
}
#Emotions to observe(all emotions except neutral)
observed_emotions=['neutral', 'happy', 'sad','angry','fear', 'disgust','surprise']

## Retrieve RAVDESS dataset from File System

In [4]:
#audio file path in file system
RAV = "../datasets/RAVDESS/audio_speech_actors_01-24/"
# test to run one example 
dir_list = os.listdir(RAV+"Actor_01/")
dir_list[0:5]

['03-01-01-01-01-01-01.wav',
 '03-01-01-01-01-02-01.wav',
 '03-01-01-01-02-01-01.wav',
 '03-01-01-01-02-02-01.wav',
 '03-01-02-01-01-01-01.wav']

In [9]:
df = pd.read_csv("../datasets/data_path.csv")
df.head()

,labels,source,path
0,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
1,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
2,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
3,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...
4,male_neutral,RAVDESS,../datasets/RAVDESS/audio_speech_actors_01-24/...


In [10]:
print(df.labels.value_counts())

male_disgust      767
male_angry        767
male_happy        767
male_sad          767
male_fear         767
male_neutral      719
female_sad        696
female_fear       696
female_happy      696
female_disgust    696
female_angry      696
female_neutral    656
Name: labels, dtype: int64


In [7]:
#Load the data and extract features for each sound file
def load_data(test_size=0.25):
    x,y=[],[]
    for file in glob.glob(RAV+"Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

In [11]:
df = pd.DataFrame(columns=['feature'])

# loop feature extraction over the entire dataset
counter=0
for index,path in enumerate(ref.path):
    X, sample_rate = librosa.load(path
                                  , res_type='kaiser_fast'
                                  ,duration=2.5
                                  ,sr=44100
                                  ,offset=0.5
                                 )
    sample_rate = np.array(sample_rate)
    
    # mean as the feature. Could do min and max etc as well. 
    mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                        sr=sample_rate, 
                                        n_mfcc=40),
                    axis=0)
    df.loc[counter] = [mfccs]
    counter=counter+1   

# Check a few records to make sure its processed successfully
print(len(df))
df.head()

8690


,feature
0,"[-21.385315, -21.385315, -21.385315, -21.38531..."
1,"[-19.669842, -19.838392, -20.179165, -18.40570..."
2,"[-21.391132, -21.391132, -21.391132, -21.39113..."
3,"[-21.469006, -21.469006, -21.469006, -21.46900..."
4,"[-22.837025, -22.837025, -22.837025, -22.83702..."


In [12]:
df = pd.concat([df,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df[:5]

,feature,0,1,2,3,4,5,6,7,8,...,206,207,208,209,210,211,212,213,214,215
0,"[-21.385315, -21.385315, -21.385315, -21.38531...",-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,...,-17.314789,-18.879824,-20.803364,-21.046108,-20.095325,-20.280087,-20.763123,-19.553648,-20.154835,-19.897285
1,"[-19.669842, -19.838392, -20.179165, -18.40570...",-19.669842,-19.838392,-20.179165,-18.405704,-18.904381,-19.933283,-20.974010,-21.251734,-21.251734,...,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734
2,"[-21.391132, -21.391132, -21.391132, -21.39113...",-21.391132,-21.391132,-21.391132,-21.391132,-21.381531,-21.391132,-21.391132,-21.254810,-20.585144,...,-21.391132,-21.391132,-21.391132,-21.391132,-21.108055,-21.259764,-21.391132,-21.391132,-21.391132,-21.391132
3,"[-21.469006, -21.469006, -21.469006, -21.46900...",-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,...,-17.526384,-17.198935,-17.701351,-18.673058,-19.091045,-19.486828,-20.428762,-20.331993,-19.380220,-18.689539
4,"[-22.837025, -22.837025, -22.837025, -22.83702...",-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,...,-18.009171,-17.820364,-18.525299,-18.332527,-17.388416,-17.585821,-19.182173,-19.070602,-18.440105,-18.917660


In [13]:
df.head()

,feature,0,1,2,3,4,5,6,7,8,...,206,207,208,209,210,211,212,213,214,215
0,"[-21.385315, -21.385315, -21.385315, -21.38531...",-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,...,-17.314789,-18.879824,-20.803364,-21.046108,-20.095325,-20.280087,-20.763123,-19.553648,-20.154835,-19.897285
1,"[-19.669842, -19.838392, -20.179165, -18.40570...",-19.669842,-19.838392,-20.179165,-18.405704,-18.904381,-19.933283,-20.974010,-21.251734,-21.251734,...,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734
2,"[-21.391132, -21.391132, -21.391132, -21.39113...",-21.391132,-21.391132,-21.391132,-21.391132,-21.381531,-21.391132,-21.391132,-21.254810,-20.585144,...,-21.391132,-21.391132,-21.391132,-21.391132,-21.108055,-21.259764,-21.391132,-21.391132,-21.391132,-21.391132
3,"[-21.469006, -21.469006, -21.469006, -21.46900...",-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,...,-17.526384,-17.198935,-17.701351,-18.673058,-19.091045,-19.486828,-20.428762,-20.331993,-19.380220,-18.689539
4,"[-22.837025, -22.837025, -22.837025, -22.83702...",-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,...,-18.009171,-17.820364,-18.525299,-18.332527,-17.388416,-17.585821,-19.182173,-19.070602,-18.440105,-18.917660


In [14]:
df=df.fillna(0)
print(df.shape)
df[:5]

(8690, 217)


,feature,0,1,2,3,4,5,6,7,8,...,206,207,208,209,210,211,212,213,214,215
0,"[-21.385315, -21.385315, -21.385315, -21.38531...",-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,-21.385315,...,-17.314789,-18.879824,-20.803364,-21.046108,-20.095325,-20.280087,-20.763123,-19.553648,-20.154835,-19.897285
1,"[-19.669842, -19.838392, -20.179165, -18.40570...",-19.669842,-19.838392,-20.179165,-18.405704,-18.904381,-19.933283,-20.974010,-21.251734,-21.251734,...,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734,-21.251734
2,"[-21.391132, -21.391132, -21.391132, -21.39113...",-21.391132,-21.391132,-21.391132,-21.391132,-21.381531,-21.391132,-21.391132,-21.254810,-20.585144,...,-21.391132,-21.391132,-21.391132,-21.391132,-21.108055,-21.259764,-21.391132,-21.391132,-21.391132,-21.391132
3,"[-21.469006, -21.469006, -21.469006, -21.46900...",-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,-21.469006,...,-17.526384,-17.198935,-17.701351,-18.673058,-19.091045,-19.486828,-20.428762,-20.331993,-19.380220,-18.689539
4,"[-22.837025, -22.837025, -22.837025, -22.83702...",-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,-22.837025,...,-18.009171,-17.820364,-18.525299,-18.332527,-17.388416,-17.585821,-19.182173,-19.070602,-18.440105,-18.917660


In [15]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['path','labels','source'],axis=1)
                                                    , df.labels
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=42
                                                   )

# Lets see how the data present itself before normalisation 
X_train[150:160]

KeyError: "['path' 'labels' 'source'] not found in axis"

In [ ]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# Check the dataset now 
X_train[150:160]

In [ ]:
max_data = np.max(X_train)
min_data = np.min(X_train)
X_train = (X_train-min_data)/(max_data-min_data+1e-6)
X_train =  X_train-0.5

max_data = np.max(X_test)
min_data = np.min(X_test)
X_test = (X_test-min_data)/(max_data-min_data+1e-6)
X_test =  X_test-0.5

X_train[150:160]

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)
#print(y_train[0:10])
#print(y_test[0:10])

# Pickel the lb object for future use 
filename = 'labels'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()

In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

## MLP Classifier and Fit/Train the Model

In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Fit/Train the model
model.fit(x_train,y_train)

## Predict the accuracy and result

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
#Calculate the accuracy of the model

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
matrix = confusion_matrix(y_test,y_pred)
#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

#Print the report
print("Statistics:")
print(classification_report(y_test,y_pred))

#Print the Confusion Matrix
print("Confusion Matrix:")
df_cm = pd.DataFrame(matrix)
sn.heatmap(df_cm, annot=True, fmt='')
plt.show()